In [61]:
import cv2
import numpy as np

# Globals
selected_colors_hsv = []
zoom_scale = 1.0
pan_offset = [0, 0]
is_panning = False
last_mouse_pos = None

# Mouse callback
def on_mouse(event, x, y, flags, param):
    global selected_colors_hsv, zoom_scale, pan_offset, is_panning, last_mouse_pos

    frame_bgr = param['frame']
    frame_hsv = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2HSV)
    h, w = frame_bgr.shape[:2]

    if event == cv2.EVENT_LBUTTONDOWN:
        frame_h, frame_w = frame_bgr.shape[:2]
        zoomed_w, zoomed_h = int(frame_w / zoom_scale), int(frame_h / zoom_scale)
        
        # Correct pan boundaries
        x_offset = min(max(pan_offset[0], 0), max(frame_w - zoomed_w, 0))
        y_offset = min(max(pan_offset[1], 0), max(frame_h - zoomed_h, 0))
        
        # Map display coordinates to cropped region
        img_x = int(x_offset + x / frame_w * zoomed_w)
        img_y = int(y_offset + y / frame_h * zoomed_h)

        if 0 <= img_x < w and 0 <= img_y < h:
            hsv_color = frame_hsv[img_y, img_x]
            selected_colors_hsv.append(hsv_color)
            print(f"Selected HSV color at ({img_x}, {img_y}): {hsv_color}")

    elif event == cv2.EVENT_MOUSEWHEEL:
        zoom_change = 0.1 if flags > 0 else -0.1
        zoom_scale = max(0.2, min(5.0, zoom_scale + zoom_change))

    elif event == cv2.EVENT_MBUTTONDOWN:
        is_panning = True
        last_mouse_pos = (x, y)

    elif event == cv2.EVENT_MOUSEMOVE and is_panning:
        dx = x - last_mouse_pos[0]
        dy = y - last_mouse_pos[1]
        pan_offset[0] = max(0, pan_offset[0] - dx)
        pan_offset[1] = max(0, pan_offset[1] - dy)
        last_mouse_pos = (x, y)

    elif event == cv2.EVENT_MBUTTONUP:
        is_panning = False

# Compute HSV bounds
def calculate_hsv_bounds(hsv_colors, margin=(10, 50, 50)):
    hsv_colors = np.array(hsv_colors)
    min_hsv = np.clip(hsv_colors.min(axis=0) - margin, [0, 0, 0], [179, 255, 255])
    max_hsv = np.clip(hsv_colors.max(axis=0) + margin, [0, 0, 0], [179, 255, 255])
    return min_hsv, max_hsv

# Apply zoom and pan
def zoom_and_crop(image, zoom_scale, pan_offset):
    h, w = image.shape[:2]
    new_w, new_h = int(w / zoom_scale), int(h / zoom_scale)
    x, y = pan_offset[0], pan_offset[1]
    x = min(max(x, 0), max(w - new_w, 0))
    y = min(max(y, 0), max(h - new_h, 0))
    cropped = image[y:y + new_h, x:x + new_w]
    return cv2.resize(cropped, (w, h), interpolation=cv2.INTER_LINEAR)

def main():
    video_path = "test2.mp4"  # 🔁 Replace with your video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file.")
        return

    ret, first_frame = cap.read()
    if not ret:
        print("Couldn't read first frame.")
        return

    clone = first_frame.copy()
    param = {'frame': clone}
    cv2.namedWindow("Select Pixels")
    cv2.setMouseCallback("Select Pixels", on_mouse, param=param)

    print("🖱️ Left-click to select pixels (HSV). Wheel to zoom, middle-drag to pan. Press 'q' when done.")
    while True:
        display = zoom_and_crop(clone.copy(), zoom_scale, pan_offset)
        cv2.putText(display, f"{len(selected_colors_hsv)} colors selected", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Select Pixels", display)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyWindow("Select Pixels")

    if not selected_colors_hsv:
        print("No colors selected. Exiting.")
        return

    lower_hsv, upper_hsv = calculate_hsv_bounds(selected_colors_hsv)
    print(f"HSV Lower Bound: {lower_hsv}, Upper Bound: {upper_hsv}")

    print("🎞️ Starting video filtering...")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lower_hsv, upper_hsv)
        result = cv2.bitwise_and(frame, frame, mask=mask)

        cv2.imshow("Filtered Video", result)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


🖱️ Left-click to select pixels (HSV). Wheel to zoom, middle-drag to pan. Press 'q' when done.
Selected HSV color at (406, 234): [175  69 184]
Selected HSV color at (407, 234): [175  74 172]
Selected HSV color at (409, 235): [176  62 192]
Selected HSV color at (407, 235): [175  67 190]
Selected HSV color at (408, 234): [176  80 150]
HSV Lower Bound: [165  12 100], Upper Bound: [179 130 242]
🎞️ Starting video filtering...


In [ ]:
q